In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Input
from keras.models import Sequential

In [5]:
image_1_path="/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"
image_2_path="/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor"
image_3_path="/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor"
image_4_path="/kaggle/input/brain-tumor-classification-mri/Training/no_tumor"

In [6]:
image_1_data=[]
image_1_storage = [os.path.join(image_2_path,i) for i in os.listdir(image_2_path)]
image_cv = [ cv2.imread(file) for file in image_1_storage]
# image_array=np.array(image_cv)
for image in image_cv:
    if image is not None and image.size > 0:
        resize_image=cv2.resize(image, (250,250))
        image_1_data.append(resize_image)
        image_array=np.array(image_1_data)

In [7]:
image_array.shape

(822, 250, 250, 3)

In [8]:
def image_convert(image_path):
    import numpy as np
    import cv2
    image_1_data=[]
    image_1_storage = [os.path.join(image_path,i) for i in os.listdir(image_path)]
    image_cv = [ cv2.imread(file) for file in image_1_storage]
    for image in image_cv:
        if image is not None and image.size > 0:
            resize_image=cv2.resize(image, (250,250))
            image_1_data.append(resize_image)
            image_array=np.array(image_1_data)
    return image_array

In [9]:
image_1_data1=image_convert(image_1_path)
image_2_data1=image_convert(image_2_path)
image_3_data1=image_convert(image_3_path)
image_4_data1=image_convert(image_4_path)

In [10]:
combined_image_data_con = np.concatenate((image_1_data1, image_2_data1, image_3_data1, image_4_data1), axis=0)
print(combined_image_data_con.shape)
print(len(combined_image_data_con))

(2870, 250, 250, 3)
2870


In [11]:
# combined_image_data = np.stack((image_1_data1, image_2_data1, image_3_data1, image_4_data1), axis=0)

# print(combined_image_data.shape) 
# print(len(combined_image_data))# (4, 826, 250, 250, 3)

In [13]:
X = combined_image_data_con.reshape(-1, 250, 250, 3)  # New shape (3304, 250, 250, 3)

# Generate labels
Y = np.array([[i]*826 for i in range(4)]).flatten()  # Ne

In [37]:
X1 = combined_image_data_con[:, :-1]  # All columns except the last one
Y1 = combined_image_data_con[:, -1] 

In [38]:
len(Y1)

2870

In [39]:
print(X1.shape)
print(Y1.shape)

(2870, 249, 250, 3)
(2870, 250, 3)


In [ ]:
# X=combined_image_data_con[:,: 3]
# Y=combined_image_data_con[:,3]

In [ ]:
print(len(X))
print(X.size)
print(X.shape)

In [ ]:
print(len(Y))
print(Y.size)

In [40]:
X_train,X_test,Y_train,Y_test=train_test_split(X1,Y1,random_state=42)

In [63]:
print(Y_train.shape)
print(Y_test.shape)

(2152, 250, 3)
(718, 250, 3)


In [59]:
Y_train_r = np.expand_dims(Y_train, axis=1)
Y_test_r = np.expand_dims(Y_test, axis=1)

In [60]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input

# Assuming X_train and X_test are numpy arrays of shape (num_samples, 249, 250, 3)
# Assuming Y_train and Y_test are numpy arrays of shape (num_samples,)

# Create Sequential model
model123 = Sequential([
    Input(shape=(249, 250, 3)),
    Conv2D(32, (3, 3), activation="relu"),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (5, 5), activation="relu"),
    Conv2D(128, (7, 7), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(512, (5, 5), activation="relu"),
    Conv2D(128, (7, 7), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(1024, (7, 7), activation="relu"),
    Conv2D(512, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(2000, activation="relu"),
    Dense(1000, activation="relu"),
    Dropout(0.25),
    Dense(2500, activation="relu"),
    Dense(500, activation="relu"),
    Dropout(0.25),
    Dense(150, activation="relu"),
    Dense(1, activation="sigmoid")  # Use 'sigmoid' for binary classification
])

# Compile the model
model123.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
model123.fit(X_train,Y_train,epochs=50,batch_size=32,verbose=1,validation_data=(X_test,Y_test),validation_split=0.2)

In [55]:
model12=Sequential([
    Input(shape=(249, 250, 3)),
    Conv2D(32,(3,3),activation="relu"),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,(5,5),activation="relu"),
    Conv2D(128,(7,7),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(512,(5,5),activation="relu"),
    Conv2D(128,(7,7),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(1024,(7,7),activation="relu"),
    Conv2D(512,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(2000,activation="relu"),
    Dense(1000,activation="relu"),
    Dropout(0.25),
    Dense(2500,activation="relu"),
    Dense(500,activation="relu"),
    Dropout(0.25),
    Dense(150,activation="relu"),
    Dense(1,activation="softmax")
])
model12.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
model12.fit(X_train,Y_train,epochs=50,batch_size=32,verbose=1,validation_data=(X_test,Y_test),validation_split=0.2)

In [ ]:
# model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
# model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Reshape input data to treat frames as channels
# X_train_reshaped = X_train.reshape(-1, 750, 3)  # Shape: (2478, 750, 3)
# X_test_reshaped=X_test.reshape(-1,750,3)
# Create Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3),padding="same"),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu',padding="same"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
                ])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, verbose=1)
# print(X_train_reshaped.shape)

In [ ]:
# X_test_reshaped1.shape

In [ ]:
# model.fit(X_train_reshaped1,Y_train,epochs=100,batch_size=32,verbose=1,validation_data=(X_test_reshaped1,Y_test),callbacks=[early_stopping],validation_split=0.2)


In [ ]:
loss,accuracy=model.evaluate(X_test_reshaped1,Y_test)
print("Loss of Sequential model:",loss)
print("accuracy of Sequential model:",accuracy)